In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import category_encoders as ce
import math

In [ ]:
#ucitavanje izvornih podataka
cols = range(1,27)
data = pd.read_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\sample_all.csv', usecols=cols)
data.info(verbose=True)

In [ ]:
#vremenski atributi
cols_to_change = ['VRIJEME_KREIRANJA', 'VRIJEME_ZAMRZAVANJA', 'DATUM_OD', 'DATUM_DO', 'VRIJEME', 'DATUM_STORNA']
date_start = pd.to_datetime("2000-01-01 00:00:00")
data['DATUM_STORNA'].fillna(0, inplace=True)

In [ ]:
for col in cols_to_change:
    data[col] = pd.to_datetime(data[col])

In [ ]:
for col in cols_to_change:
    data['WEEKDAY_'+col] = data[col].dt.dayofweek
    data['WEEK_'+col] = data[col].dt.week
    data['MONTH_'+col] = data[col].dt.month
    data[col] = (data[col]-date_start).astype('timedelta64[D]')

In [ ]:
data = data.drop(['SIF_DRZAVE.1', 'WEEKDAY_DATUM_STORNA', 'WEEK_DATUM_STORNA', 'MONTH_DATUM_STORNA'], axis=1)
data = data.dropna()

In [ ]:
#ciklicki atributi
cols_to_change = ['VRIJEME_KREIRANJA', 'VRIJEME_ZAMRZAVANJA', 'DATUM_OD', 'DATUM_DO', 'VRIJEME']
for col in cols_to_change:
    weekday = 'WEEKDAY_'+col
    week = 'WEEK_'+col
    month = 'MONTH_'+col
    subset = [weekday, week, month]
    for v in subset:
        maxx = data[v].max()
        data['SIN_'+v] = np.sin((2*np.pi*data[v])/maxx)
        data['COS_'+v] = np.cos((2*np.pi*data[v])/maxx)

In [ ]:
col_drop = ['WEEKDAY_VRIJEME_KREIRANJA', 'WEEK_VRIJEME_KREIRANJA', 'MONTH_VRIJEME_KREIRANJA', 
            'WEEKDAY_VRIJEME_ZAMRZAVANJA', 'WEEK_VRIJEME_ZAMRZAVANJA', 'MONTH_VRIJEME_ZAMRZAVANJA', 
            'WEEKDAY_DATUM_OD', 'WEEK_DATUM_OD', 'MONTH_DATUM_OD', 'WEEKDAY_DATUM_DO', 
            'WEEK_DATUM_DO', 'MONTH_DATUM_DO', 'WEEKDAY_VRIJEME', 'WEEK_VRIJEME', 'MONTH_VRIJEME',
            'SIN_WEEKDAY_VRIJEME', 'COS_WEEKDAY_VRIJEME', 'SIN_WEEK_VRIJEME', 'COS_WEEK_VRIJEME', 
            'SIN_MONTH_VRIJEME', 'COS_MONTH_VRIJEME']
data = data.drop(col_drop, axis=1)

In [ ]:
cols_dummy = data[['OBRISANO', 'OBAVEZA_AKONTACIJE', 'STORNO', 'KANAL_ID', 'STATUS_REZERVACIJE']]
prefixes = ['OBRISANO', 'OBAVEZA_AKONTACIJE', 'STORNO', 'KANAL', 'STAT_REZERVACIJE']
data_dummy = pd.get_dummies(cols_dummy, prefix = prefixes, prefix_sep='_')

In [ ]:
data=pd.merge(data, data_dummy, 'left', left_index=True, right_index=True)
data = data.drop(['OBRISANO', 'OBAVEZA_AKONTACIJE', 'STORNO', 'KANAL_ID', 'STATUS_REZERVACIJE'], axis=1)
data = data.drop(['STORNO_NE', 'OBAVEZA_AKONTACIJE_NE', 'OBRISANO_NE', 'KANAL', 'VRSTA_SOBE'], axis = 1)

In [ ]:
data['DATUM_STORNA'] = data['DATUM_STORNA'].replace(-10957.0, 0.0)

In [ ]:
data['TIP_SOBE'] = data['TIP_SOBE'].astype('category')
data['SIF_DRZAVE'] = data['SIF_DRZAVE'].astype('category')
cat_col = data.select_dtypes(['category']).columns
data[cat_col] = data[cat_col].apply(lambda x: x.cat.codes)

In [ ]:
data['DELTA_REZERVACIJA'] = data['DATUM_OD'] - data['VRIJEME_KREIRANJA']
data['DELTA_REZERVACIJA'][data['DELTA_REZERVACIJA']<0.0]=0.0
data['DELTA_STORNO_TOCH'] = data['DATUM_OD']-data['DATUM_STORNA']
data['DELTA_STORNO_TOCH'][data['DATUM_STORNA']==0.0] = 0.0
data = data[data['DELTA_STORNO_TOCH']>=0.0]

In [ ]:
data['DELTA_STORNO_FROMREZ'] = data['DATUM_STORNA'] - data['VRIJEME_KREIRANJA']
data['DELTA_STORNO_FROMREZ'][data['DATUM_STORNA']==0.0] = 0.0
data = data[data['DELTA_STORNO_FROMREZ']>=0.0]

In [ ]:
#kategorizacija izlaza
bins = [-300.0, 0.0, 20.0, 50.0, 120.0, 170.0, 220.0, 300.0, 900.0]
labels = [0, 1, 2, 3, 4, 5, 6, 7]
data['Y_CAT_STORNO_TOCH'] = pd.cut(data['DELTA_STORNO_TOCH'], bins=bins, labels=labels, include_lowest=True)
bins = [-300.0, 0.0, 30.0, 80.0, 130.0, 215.0, 290.0, 900.0]
labels = [0, 1, 2, 3, 4, 5, 6]
data['Y_CAT_STORNO_FROMREZ'] = pd.cut(data['DELTA_STORNO_FROMREZ'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
col_drop = ['VRIJEME_KREIRANJA', 'VRIJEME_ZAMRZAVANJA', 'DATUM_OD', 'DATUM_DO', 'DATUM_STORNA' ]
data = data.drop(col_drop, axis=1)

In [ ]:
#spasavanje dataseta
data.to_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\final_dataset\\dataset.csv')